# Understanding Relational Graph Convolutional Networks (R-GCNs)
What happens under the hood of Graph Neural Networks (GNNs) applied to multi-relational data, such as Knowledge Graphs (KGs)? A brief introduction to R-GCNs using pure numpy.

Original Paper: Schlichtkrull, M., Kipf, T. N., Bloem, P., Van Den Berg, R., Titov, I., & Welling, M. (2018, June). *Modeling relational data with graph convolutional networks*. In *European Semantic Web Conference* (pp. 593-607). Springer, Cham. 

## Requirements
- Numpy.


In [1]:
try:
    import numpy as np
except ImportError as e:
    print('numpy is not available in you environment: try "pip install numpy"')

np.random.seed(1)

## Recall on the GNNs (and the Vanilla GCNs)
This section provides a recall on the behaviour of a basic GNN layer. 

Main ingredients:
- One-hot vectors (no features) adopted to represent nodes.
- Weight matrix representing the learnable parameters (or weights).
- Adjacency matrix describing undirected edges between nodes. 

In [2]:
import numpy as np

# One-hot vectors for representing nodes (randomly initialized)
X = np.eye(5, 5)
n = X.shape[0]
np.random.shuffle(X)

print('\n\n----- One-hot vector representation of nodes:')
print(X)

# Low-dimensional vector to represent node embeddings
emb = 3 

# Weight matrix (randomly inizialized according to Glorot and Bengio (2010))
W = np.random.uniform(-np.sqrt(1. / emb), np.sqrt(1. / emb), (n, emb))

print('\n\n----- Weight Matrix:')
print(W)

# Adjacency matrix (randomly initialized)
A = np.random.randint(2, size=(n, n))
np.fill_diagonal(A, 1)  # Include the self loop
A_und = (A + A.T) # Hack for creating a symmetric adjacency matrix (undirected graph)
A_und[A_und > 1] = 1

print('\n\n----- Adjacency Matrix (undirected graph):')
print(A_und)



----- One-hot vector representation of nodes:
[[0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]]


----- Weight Matrix:
[[-0.4294049   0.57624235 -0.3047382 ]
 [-0.11941829 -0.12942953  0.19600584]
 [ 0.5029172   0.3998854  -0.21561317]
 [ 0.02834577 -0.06529497 -0.31225734]
 [ 0.03973776  0.47800217 -0.04941563]]


----- Adjacency Matrix (undirected graph):
[[1 1 1 0 1]
 [1 1 1 1 1]
 [1 1 1 1 0]
 [0 1 1 1 1]
 [1 1 0 1 1]]


Considering these ingredients, a "recursive neighborhood diffusion" is performed through the so-called “message passing framework”. The main idea behind this framework is that each node representation is updated with its neighbors' features. The neighbors' features are *passed* to the target node as *messages* through the edges. 

The operations are the following:
* A linear transformation (or projection) involving the nodes features and the weight matrix.
* A neighborhood diffusion to update the nodes representations, aggregating the features of its neighbors. 

In [3]:
# Linear transformation
L_0 = X.dot(W)

print('\n\n----- Output of the linear transformation:')
print(L_0)

# Neighborhood diffusion
ND_GNN = A_und.dot(L_0)

print('\n\n----- (GNN) Output of the neighborhood diffusion:')
print(ND_GNN)

# Test on the aggregation:
assert(ND_GNN[0,0] == L_0[0,0] + L_0[1,0] + L_0[2,0] + L_0[4,0])




----- Output of the linear transformation:
[[ 0.5029172   0.3998854  -0.21561317]
 [-0.11941829 -0.12942953  0.19600584]
 [ 0.03973776  0.47800217 -0.04941563]
 [-0.4294049   0.57624235 -0.3047382 ]
 [ 0.02834577 -0.06529497 -0.31225734]]


----- (GNN) Output of the neighborhood diffusion:
[[ 0.45158244  0.68316307 -0.3812803 ]
 [ 0.02217754  1.25940542 -0.6860185 ]
 [-0.00616823  1.3247004  -0.37376116]
 [-0.48073966  0.85952002 -0.47040533]
 [-0.01756022  0.78140325 -0.63660287]]


In the simplest formulation of the GNN, represented by the Vanilla Graph Convolutional Networks (GCNs), the aggregation/update operation is an isotropic computation, where the features of neighbor nodes are considered in the same way. 

More precisely, an isotropic *average* computation is performed in the specific case of Vanilla GCNs. This average operation requires a new ingredient represented by the indegree of each node, which consists in the number of incoming edges.

In [4]:
# Degree vector (degree for each node)
D = A_und.sum(axis=1)

print('\n\n----- Degree vector - Each element represents the i-node degree:')
print(D)

# Reciprocal of the degree to perform the average computation (diagonal matrix)
D_rec = np.diag(np.reciprocal(D.astype(np.float32))) # Need to convert degree integer values as float

print('\n\n----- Reciprocal of the degree (in a diagonal matrix):')
print(D_rec)

# Isotropic average computation
ND_GCN = D_rec.dot(ND_GNN)

print('\n\n----- (GCN) Isotropic average computation:')
print(ND_GCN)

# Test on the isotropic average computation:
assert(ND_GCN[0,0] == ND_GNN[0,0] * D_rec[0,0])



----- Degree vector - Each element represents the i-node degree:
[4 5 4 4 4]


----- Reciprocal of the degree (in a diagonal matrix):
[[0.25 0.   0.   0.   0.  ]
 [0.   0.2  0.   0.   0.  ]
 [0.   0.   0.25 0.   0.  ]
 [0.   0.   0.   0.25 0.  ]
 [0.   0.   0.   0.   0.25]]


----- (GCN) Isotropic average computation:
[[ 0.11289561  0.17079077 -0.09532007]
 [ 0.00443551  0.25188109 -0.1372037 ]
 [-0.00154206  0.3311751  -0.09344029]
 [-0.12018491  0.21488001 -0.11760133]
 [-0.00439005  0.19535081 -0.15915072]]


## From GCNs to R-GCNs for encoding KGs
The previous example considers an undirected and no-typed graph. As mentioned before, the update process is based on the following steps (the node indegree is not considered for the sake of simplicity):

1. a projection step (or linear transformation), which is achieved multiplying of: (i) the one-hot feature matrix with (ii) the weight matrix.
    
    (i). Matrix (n, n) that defines the initial features of the nodes.
    
    (ii). Matrix (n, emb) that describes the model's parameters. The current matrix is able to encode only one type of relation.

2. an aggregation step, which is achieved multiplying: (i) the adjacency matrix with (ii) the matrix resulting from the projection step.

    (i). Symmetric Matrix (n, n) that describes undirected and untyped edges.

    (ii). Matrix (n, emb) that describes the latent node representation of nodes.

In order to extend the GCN layer to encode the structure of a KG, we need to represent our data as a directed and multi-typed graph. The update process is similar to the previous one, but the ingredients are more complex:

1. a projection step, which is achieved multiplying: (i) the one-hot feature matrix with (ii) the weight **tensor**.
    
    (i). Matrix (n, n) that defines the initial features of the nodes.
    
    (ii). **Tensor (r, n, emb)** that describes the model's parameters, which will embed the latent node representations at the end of the training process. This tensor is able to encode different relations by stacking **r** batches of matrices (n, emb). Each of these batches encodes a single typed relation.

Tip: the projection step will no longer be a simple multiplication of matrices, but it will be a *batch matrix multiplication*, in which (i) is multiplied with each batch of (ii).

2. an aggregation step, which is achieved multiplying (i) the **(directed) adjacency tensor** with (ii) the **tensor** resulting from the projection step.

    (i) **Tensor (r, n, n)** that describes directed and **r**-typed edges. This tensor is composed of **r** batches of adjacency matrices (n,n). In detail, each of these matrices describes the edges between nodes, according to a specific type of relation. Moreover, compared to the adjacency matrix of an undirected graph, each of these adjacency matrices is not symmetric, because it encodes a specific edge direction.
    (ii) **Tensor (r, n, emb)** is the result of the projection layer.

Tip: as happened for the projection step, the aggregation phase consists of a *batch matrix multiplication*. Each batch of (i) is multiplied with each batch of (ii). This aggregation defines the GCN transformation for each batch. At the end of the process, the batches have to be added together (R-GCN) to obtain a node representation that incorporates the neighborhood aggregation according to different type of relations.

The following example shows the behaviour of a R-GCN layer encoding a directed and multi-typed graph with 2 types of edges (or relations).


In [5]:
print('\n\n----- Recall --> One-hot vector representation of nodes:')
print(X)

# Number of relation types
num_rels = 2

print('\n\n----- Number of relation types:')
print(num_rels)

# Weight matrix of relation number 1 (randomly inizialized according to Glorot and Bengio (2010))
W_rel1 = np.random.uniform(-np.sqrt(1. / emb), np.sqrt(1. / emb), (n, emb))
print('\n\n----- Weight matrix of relation 1:')
print(W_rel1)

# Weight matrix of relation number 2 (randomly initialized with uniform distribution)
W_rel2 = np.random.uniform(1/100, 0.5, (n, emb))
print('\n\n----- Weight matrix of relation 2:')
print(W_rel2)

# Tensor including both weight matrices
W_rels =  np.concatenate((W_rel1, W_rel2))
W_rels = np.reshape(W_rels,(num_rels, n, emb)) # num_rels is the number of the relations, n is the number of nodes, emb is the low-dimensional representation
print('\n\n-----Tensor including both weight matrices:')
print(W_rels)

L_0_rels = np.matmul(X, W_rels)
print('\n\n----- Linear trasformation (or projection) with batch matrix multiplication:')
print(L_0_rels)

# Adjacency matrix of relation number 1
A_rel1 = np.random.randint(2, size=(n, n))
np.fill_diagonal(A, 0)  # Not consider the self loop (diag values = 0)
print('\n\n----- Adjacency matrix of relation 1:')
print(A_rel1)

# Adjacency matrix of relation number 2
A_rel2 = np.random.randint(3,size=(n,n))
np.fill_diagonal(A_rel2, 0)  # Not consider the self loop (diag values = 0)
A_rel2[A_rel2>1] = 0
print('\n\n----- Adjacency matrix of relation 2:')
print(A_rel2)

# Tensor including both adjacency matrices
A_rels = np.concatenate((A_rel1, A_rel2))
A_rels = np.reshape(A_rels, (num_rels, n, n)) # num_rels is the number of the relations, (n,n) is the dimension of the original adj matrix
print('\n\n----- Tensor including both adjacency matrices:')
print(A_rels)

# GCN for each typed edge
ND_GCN = np.matmul(A_rels, L_0_rels)
print('\n\n----- (GCN) Output of the neighborhood diffusion (for each typed edge):')
print(ND_GCN)

# R-GCN
RGCN = np.sum(ND_GCN, axis=0)
print('\n\n----- (R-GCN) Aggregation of the results of the GCN layer applied to different types of edge:')
print(RGCN)

# Test of the aggregation
assert(RGCN[0,0] == L_0_rels[0,1,0] + L_0_rels[0,2,0] + L_0_rels[0,3,0] + L_0_rels[0,4,0] + L_0_rels[1,3,0])







----- Recall --&gt; One-hot vector representation of nodes:
[[0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]]


----- Number of relation types:
2


----- Weight matrix of relation 1:
[[-0.46378913 -0.09109707  0.52872529]
 [ 0.03829597  0.22156061 -0.2130242 ]
 [ 0.21535272  0.38639244 -0.55623279]
 [ 0.28884178  0.56448816  0.28655701]
 [-0.25352144  0.334031   -0.45815514]]


----- Weight matrix of relation 2:
[[0.22946783 0.4552118  0.15387093]
 [0.15100992 0.073714   0.01948981]
 [0.34262941 0.11369778 0.14011786]
 [0.25087085 0.03614765 0.29131763]
 [0.081897   0.29875971 0.3528816 ]]


-----Tensor including both weight matrices:
[[[-0.46378913 -0.09109707  0.52872529]
  [ 0.03829597  0.22156061 -0.2130242 ]
  [ 0.21535272  0.38639244 -0.55623279]
  [ 0.28884178  0.56448816  0.28655701]
  [-0.25352144  0.334031   -0.45815514]]

 [[ 0.22946783  0.4552118   0.15387093]
  [ 0.15100992  0.073714    0.01948981]
  [ 0.34262941  0.11369778  0.14